In [ ]:
import pandas as pd
%run ./libs.ipynb

In [ ]:
data = pd.read_csv("./data/Airbnb_Open_Data.csv", low_memory=False)

In [ ]:
print_details(data)

In [ ]:
del data['license']

## house_rules

In [ ]:
data['missing_house_rules'] = data['house_rules'].isna()

In [ ]:
data['missing_house_rules'].head()

In [ ]:
data['clean_house_rules'] = preprocess(data['house_rules'], False)
del data['house_rules']
del data['clean_house_rules']

In [ ]:
print_details(data)

### last review

In [ ]:
data['clean_last_review'] = pd.to_datetime(data['last review'], format='%m/%d/%Y')

In [ ]:
data['clean_last_review'].head()

In [ ]:
del data['last review']
data.sort_values(by='clean_last_review', ascending=False, inplace=True)

In [ ]:
data = data[data['clean_last_review']<'2022-09-13']
data = data[data['clean_last_review']>'2014-12-31']

In [ ]:
df = data[['clean_last_review', 'id']].groupby('clean_last_review').count().reset_index()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style = "darkgrid")
sns.lineplot(x='clean_last_review', y='id', data=df)
plt.show()

In [ ]:
print_details(data)

### remove all other missing values

In [ ]:
data = data.dropna()

In [ ]:
data.shape

### working with duplicates

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace = True, keep = 'first')

In [ ]:
print_details(data)

In [ ]:
data['id'].duplicated().sum()


## video 2

In [ ]:
from IPython.display import display
pd.options.display.max_columns = None

In [ ]:
data["id"].unique().shape

In [ ]:
data.shape

### NAME

In [ ]:
data['clean_accomodation_bio'] = preprocess(data['NAME'], False)
del data['NAME']

### host id

In [ ]:
data["host id"].unique().shape
del data['host id']

### host_identity_verified

In [ ]:
data['host_identity_verified'] = data['host_identity_verified'].apply(host_identity_verified)

In [ ]:
del data['host name']

### neighbourhood

In [ ]:
data['neighbourhood group'].unique()

In [ ]:
data.loc[(data['neighbourhood group']=='brookln'),'neighbourhood group'] = 'Brooklyn'

In [ ]:
del data['neighbourhood']

### country

In [ ]:
del data['country']
del data['country code']

### instant_bookable

In [ ]:
data['instant_bookable'] = data['instant_bookable'].astype(bool)

In [ ]:
print_details(data)

### price and fees

In [ ]:
for col in ['price', 'service fee']:
    data[col] = data[col].apply(lambda value: float(str(value).replace('$', '').replace(',', '')))

In [ ]:
data.head()

### Construction year

In [ ]:
data['Construction year'] = data['Construction year'].apply(int)

### cancellation_policy

In [ ]:
data['cancellation_policy'].unique()

### room type

In [ ]:
data['room type'] = data['room type'].apply(lambda value: str(value).replace('/', ' '))

In [ ]:
data

In [ ]:
num_cols = list(data.select_dtypes(['int64', 'float64']).columns)
data = data[(data['minimum nights']>0)&(data['minimum nights']<365)]
for col in num_cols:
    sns.histplot(data=data, x=col)
    plt.show()

In [ ]:
data['minimum nights'].max()

In [ ]:
data['minimum nights'].min()

In [ ]:
data['minimum nights'].mode()

In [ ]:
data

In [ ]:
column_names = list(data.columns)
new_column_names = parse_column_names(column_names)

In [ ]:
data.rename(columns=new_column_names, inplace=True)

In [ ]:
data.to_csv('./data/clean_airbnb.csv', index=False)

In [ ]:
data.shape